This notebook builds an interactive figure using holoviews plot and widget objects.  It allows inspection of netcdf files shaped as time x sample x depth at a time stamp by time stamp basis.  We at USGS use this to diagnoe isntrument issues and QA/QC

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import streams # do I use this?
import hvplot.xarray
import panel as pn

# initializations
pn.extension()  # initializes the panel back end

In [ ]:
# TODO - if this is to be a demo, need to make the repo file under 2 GB

Open the data file.  We want xarray, so that we get lazy access to this file.  The holoviews package also understands xarray objects natively.

In [ ]:
# this is a well formed file for xarray (CF time, no EPIC variables) and it is 2D with sample dimension, 
# and it is also 4 GB, so we don't want to do anything that will cause the data to be read into memory
# this has already been demonstrated to crash the browser
# infile = r'C:\data\10811_V20784\python\10811whVwave03repo.cdf';
infile = '11121whVwaves02repo.cdf' # no EPIC times in here
# ds = xr.open_dataset(infile,decode_times=True, decode_cf=True, use_cftime=True) #,drop_variables=vars2omit)
# the above put out a gregorian flavor of time
# ds = xr.open_dataset(infile,decode_times=True, decode_cf=True, use_cftime=True) #,drop_variables=vars2omit)
ds = xr.open_dataset(infile) 

Helper functions here get infromation that is not otherwaise automatically understood or read by holoviews.

In [ ]:
# get a list of the variable names in the file that are not ordinate
def get_var_name_list(ds):
    coordnames = []
    for cname in ds.coords:
        coordnames.append(cname)

    varnames = []
    for vname in ds.variables:
        # eliminate any coordinates
        if vname not in coordnames:
            varnames.append(vname)
            
    return varnames

Set up the objects that will select data to display from the file.  As a test, we also optionally show each object (by calling the object on a seaprate line) to test them.  Once we know they are working as we like, we comment those lines out.

These were chosen from the gallery of widgets here:  https://panel.pyviz.org/reference/index.html

In [ ]:
# I use the helper function defined above get_var_name_list to make a list of variable names that make sense to plot
select_variable = pn.widgets.Select(options=get_var_name_list(ds),name='Pick a variable')

select_variable # test
# TODO name is not in the docs as something that can be set - is ther a generic set of thigns that can be set in all widgets?

In [ ]:
# depth_slider = pn.widgets.FloatSlider(start=ds['depth'].values.min(), end=ds['depth'].values.max(), value=ds['depth'][:].values.mean())
depth_spinner = pn.widgets.Spinner(name='Set depth to view', start=ds['depth'].values.min(), end=ds['depth'].values.max(), 
                                   value=ds['depth'][:].values.mean(), step=1)

depth_spinner  # test
# TODO the depth needs to be incremented in a more meaningful way than 1 m (e.g. step=1)

In [ ]:
# burst_slider = pn.widgets.IntSlider(start=0, end=10, value=5)
# note that get_date_input depends on this widget
burst_spinner = pn.widgets.Spinner(name='Set burst index to view', start=0, end=len(ds['time']), 
                                   value=len(ds['time'])/2, step=1)
burst_spinner # test

The next few inactive cells demonstrate the progression of how the final code was developed and what was tested.

The elegant thing about xarray is that we can now pull out this single slice of data, as shown by the code below

here I am trying out color maps and customization of the plot, information from
* http://holoviews.org/user_guide/Colormaps.html
* http://holoviews.org/user_guide/Style_Mapping.html

now select the slice of data by time and plot that, instead of indexing into the file each time

In [ ]:
burst_num = 100
burst_num = int(burst_num) # the spinner will return a float, we need an index
time_from_burst_selection = ds['time'][burst_num]

In [ ]:
data_slice = ds.sel(time=time_from_burst_selection)['Pressure']
print(len(data_slice.shape))
hv.Curve(data_slice)

We are almost there.  Here we put it all together.  The decorator depends gets the information from the widgets, and backend of the system updates the plot when a widget is changed.  

In [ ]:
@pn.depends(burst_num=burst_spinner.param.value, 
            var_name=select_variable.param.value, 
            depth=depth_spinner.param.value)
def create_plot_obj(burst_num, var_name='Pressure', depth=0, **kwargs):
    
    depth_cell = np.min(np.where(ds['depth']>=depth)) # find the equivalent depth
    burst_num = int(burst_num) # the spinner will return a float, we need an index
    time_from_burst_selection = ds['time'][burst_num]
   
    data_slice = ds.sel(time=time_from_burst_selection)[var_name]

    # build a title for the plot
    plot_title = '{} at {}, index {}'.format(data_slice.long_name, burst_num, time_from_burst_selection)
    
    # TODO check for lat and lon or ordinate dimensions other than time, depth and sample and squash them
        
    plot_width = 700
    
    if len(data_slice.shape) == 1:
        the_plot_object = hv.Curve(data_slice)
        the_plot_object.opts(width=plot_width, title=plot_title) 
        
    if len(data_slice.shape) == 2:
        
        if 'vel' in var_name: 
            cmap='seismic'
        elif 'att' in var_name: 
            cmap='PiYG'
        else: 
            cmap='Viridis'
            
        y_label = '{}, {}'.format(data_slice.depth.long_name, data_slice.depth.units)
        the_plot_object = hv.Image((data_slice['sample'].values,np.flipud(data_slice['depth'].values),
                                    data_slice.values.transpose()), 
                                    kdims=['sample',y_label])

        the_plot_object.opts(colorbar=True,cmap=cmap,width=plot_width,title=plot_title) 
        
    return the_plot_object

pn.Row(pn.Column(select_variable, burst_spinner, depth_spinner, width=180), create_plot_obj)

In [ ]:
ds.close()